In [1]:
import torch
from torch import nn
from torch.autograd import Variable
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd
from pickle import dump

In [2]:
data = pd.read_csv('train_with_out_vanishing.csv')

In [3]:
data.head()

,Positions X
0,-5216.84
1,-5216.01
2,-5213.73
3,-5210.01
4,-5204.82


In [4]:
data.shape

(624, 1)

In [5]:
data.drop_duplicates(inplace= True)

In [6]:
data.shape

(624, 1)

In [7]:
scaler = MinMaxScaler(feature_range = (0,1)) # escalando os dados
data_scaled = scaler.fit_transform(data)
dump(scaler, open('scaler.pkl', 'wb')) # salvando o scaler.

In [8]:
last_positions = [] # lista que irá armazenar as últimas 10 posições.
real_position = [] # lista que irá armazenar as posições reais.
for i in range(10, 624):
    last_positions.append(data_scaled[i-10:i, 0])
    real_position.append(data_scaled[i, 0])

In [9]:
last_positions, real_position = np.array(last_positions), np.array(real_position)

In [10]:
last_positions = np.reshape(last_positions, (last_positions.shape[0], last_positions.shape[1], 1))

In [11]:
last_positions = torch.from_numpy(last_positions).type(torch.Tensor)
real_position = torch.from_numpy(real_position).type(torch.Tensor)

In [12]:
input_dim = 1
hidden_dim = 100
num_layers = 1
output_dim = 1
num_epochs = 50

In [19]:
class LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(LSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        self.gru = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
        out, (hn, cn) = self.gru(x, (h0.detach(), c0.detach()))
        out = self.fc(out[:, -1, :]) 
        return out

In [21]:
model = LSTM(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_layers=num_layers)
criterion = torch.nn.MSELoss(reduction='mean')
optimiser = torch.optim.Adam(model.parameters(), lr=0.01)

In [22]:
import time
hist = np.zeros(num_epochs)
start_time = time.time()
lstm = []
for t in range(num_epochs):
    y_train_pred = model(last_positions)    
    loss = criterion(y_train_pred, real_position)
    print("Epoch ", t, "MSE: ", loss.item())
    hist[t] = loss.item()    
    optimiser.zero_grad()
    loss.backward()
    optimiser.step()
    
training_time = time.time() - start_time
print("Training time: {}".format(training_time))

Epoch  0 MSE:  0.287265807390213
Epoch  1 MSE:  0.13181503117084503
Epoch  2 MSE:  0.1187618300318718
Epoch  3 MSE:  0.08363600820302963
Epoch  4 MSE:  0.09280961751937866
Epoch  5 MSE:  0.09749489277601242
Epoch  6 MSE:  0.0948040783405304
Epoch  7 MSE:  0.08882716298103333
Epoch  8 MSE:  0.08381333202123642
Epoch  9 MSE:  0.08323613554239273
Epoch  10 MSE:  0.08674807846546173
Epoch  11 MSE:  0.08826617896556854
Epoch  12 MSE:  0.08588133752346039
Epoch  13 MSE:  0.08328495919704437
Epoch  14 MSE:  0.08266036212444305
Epoch  15 MSE:  0.08350104093551636
Epoch  16 MSE:  0.08455619215965271
Epoch  17 MSE:  0.0849846825003624
Epoch  18 MSE:  0.0845947340130806
Epoch  19 MSE:  0.08369915187358856
Epoch  20 MSE:  0.08284489810466766
Epoch  21 MSE:  0.08249896764755249
Epoch  22 MSE:  0.08276603370904922
Epoch  23 MSE:  0.083304263651371
Epoch  24 MSE:  0.08359523117542267
Epoch  25 MSE:  0.0833965614438057
Epoch  26 MSE:  0.08290993422269821
Epoch  27 MSE:  0.08251533657312393
Epoch  28 M

In [24]:
data_test = pd.read_csv('test_with_out_vanishing .csv')

In [25]:
data_test.head()

,Positions X
0,4680.74
1,4696.74
2,4712.74
3,4728.74
4,4744.74


In [26]:
data_test.shape

(31, 1)

In [27]:
data_completed = pd.concat((data, data_test), axis = 0)

In [28]:
data_completed.shape

(655, 1)

In [29]:
inputs = data_completed[len(data_completed) - len(data_test) - 10:].values

In [30]:
inputs = inputs.reshape(-1, 1)

In [31]:
inputs = scaler.transform(inputs)

In [32]:
X_test = []
for i in range(10, 41):
    X_test.append(inputs[i - 10:i, 0])

In [33]:
X_test = np.array(X_test)

In [34]:
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
X_test = torch.from_numpy(X_test).type(torch.Tensor)

In [35]:
model.eval()
with torch.no_grad():
    predict = model(X_test)

In [36]:
predict = scaler.inverse_transform(predict)

In [37]:
predict[0]

array([-172.7403153])

In [38]:
data_test.head()

,Positions X
0,4680.74
1,4696.74
2,4712.74
3,4728.74
4,4744.74
